In [41]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

In [14]:
# def grid(data_path):
    

# def process_data(data_path):
    
#     min_x = np.min(data['x'])
#     min_y = np.min(data['y'])
#     data = data[(data['x'] < (min_x + 0.5)) & (data['y'] < (min_y + 0.5)) ]
#     print np.shape(data)
#     print np.min(data['x']),np.min(data['y']), np.max(data['x']),np.max(data['y'])
#     place = data['place_id'].value_counts()
#     print np.shape(place)
#     print place
#     count = 0 
#     for each in place:
#         if each < 3:
#             count+=1
#     print count
#     print 




In [ ]:
if __name__ == '__main__':
    base_path = "/Users/ludai/Desktop/Github/fda/Assignment3/src/"
    x_size = 0.5
    y_size = 0.5
    #fw = [500, 1000, 4, 3, 1./22., 2, 10] #feature weights
    fw = [1, 1, 1, 1, 1, 1, 1]
    th = 5 #Keeping place_ids with more than th samples.
    for i in range(1):
        string  =  'subset%s.csv' % i
        data_path = base_path + string
        #print data_path
        df = pd.read_csv(data_path, usecols=['row_id','x','y','time','place_id'], index_col = 0)
        initial_date = np.datetime64('2014-01-01T01:01', dtype='datetime64[m]') 
        d_times = pd.DatetimeIndex(initial_date + np.timedelta64(t, 'm') for t in data.time.values)    
        df['hour'] = d_times.hour * fw[2]
        df['weekday'] = d_times.weekday * fw[3]
        df['day'] = (d_times.dayofyear * fw[4]).astype(int)
        df['month'] = d_times.month * fw[5]
        df['year'] = (d_times.year - 2013) * fw[6]
        df = df.drop(['time'], axis=1) 
        df.x = df.x.values * fw[0]
        df.y = df.y.values * fw[1]
        print np.shape(df)
        place_counts = df.place_id.value_counts()
        print np.shape(place_counts)
        #drop rows with place_id which has samples < th
        mask = (place_counts[df.place_id.values] >= th).values
        print np.shape(mask)
        df = df.loc[mask]
        #split data into training and testing sets
        df_train = df.sample(frac=0.7)
        df_test = df.loc[~df.index.isin(df_train.index)]
        print np.shape(df), np.shape(df_train), np.shape(df_test)
        
        #preparing training sets
        le = preprocessing.LabelEncoder()
        train_Y = le.fit_transform(df_train['place_id'].values)
        #print train_Y
        train_X = df_train.drop(['place_id'], axis=1).values.astype(int)
        #print train_X
     
        knn_clf = KNeighborsClassifier(n_neighbors=25, weights='distance', metric='manhattan')
        knn_clf.fit(train_X, train_Y)
        
        #preparing testing sets
        test_place = df_test['place_id'].values
        test  =  df_test.drop(['place_id'], axis =1 ).values.astype(int)
        #print np.shape(test_place), np.shape(test)
    
        test_pred = knn_clf.predict(test)
        print test_pred
        labels = le.inverse_transform(test_pred)
        print labels
        print np.shape(test_pred), np.shape(test_place)
        count = 0
        total = np.shape(labels)[0]

(1000000, 8)
(16647,)
(1000000,)
(984868, 8) (689408, 8) (295460, 8)


In [55]:
        for i in range(total):
            if labels[i]==test_place[i]:
                count+=1
        print count, float(count)/float(total)

160219 0.542269681175
